## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-03-16-51-33 +0000,nyt,Jane Fonda Revives Her Father’s McCarthy-Era F...,https://www.nytimes.com/2025/10/03/arts/jane-f...
1,2025-10-03-16-48-48 +0000,nyt,Here’s the latest.,https://www.nytimes.com/live/2025/10/03/us/tru...
2,2025-10-03-16-43-49 +0000,nypost,Slain Arizona teen kept in makeshift cage by g...,https://nypost.com/2025/10/03/us-news/death-of...
3,2025-10-03-16-40-17 +0000,nyt,Ant Yogurt Is a Traditional Recipe That Crawls...,https://www.nytimes.com/2025/10/03/science/ant...
4,2025-10-03-16-39-01 +0000,startribune,RandBall: Vikings face nightmare of a must-win...,https://www.startribune.com/minnesota-vikings-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2349/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
51,trump,41
105,attack,21
103,manchester,20
43,shutdown,19
150,synagogue,18


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
235,2025-10-02-22-15-00 +0000,wsj,President Trump intensified pressure on Democr...,https://www.wsj.com/politics/trump-to-meet-wit...,110
46,2025-10-03-15-03-00 +0000,wsj,The Trump administration has frozen $2.1 billi...,https://www.wsj.com/politics/policy/trump-admi...,103
103,2025-10-03-12-00-08 +0000,wapo,One victim in Yom Kippur synagogue attack in E...,https://www.washingtonpost.com/world/2025/10/0...,94
200,2025-10-03-01-13-09 +0000,nyt,Trump Escalates Attacks on Democrats in a Gove...,https://www.nytimes.com/2025/10/02/us/politics...,93
171,2025-10-03-07-13-06 +0000,nypost,UK police name 2 victims of Manchester synagog...,https://nypost.com/2025/10/03/world-news/uk-po...,92


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
235,110,2025-10-02-22-15-00 +0000,wsj,President Trump intensified pressure on Democr...,https://www.wsj.com/politics/trump-to-meet-wit...
103,94,2025-10-03-12-00-08 +0000,wapo,One victim in Yom Kippur synagogue attack in E...,https://www.washingtonpost.com/world/2025/10/0...
61,59,2025-10-03-14-23-50 +0000,nypost,Cops probe whether Manchester synagogue terror...,https://nypost.com/2025/10/03/world-news/manch...
209,47,2025-10-03-00-45-00 +0000,wsj,A multibillion-dollar deal to send Nvidia’s AI...,https://www.wsj.com/politics/policy/nvidia-tru...
253,46,2025-10-02-21-25-48 +0000,nypost,Ukraine pitches plan to help US sell American ...,https://nypost.com/2025/10/02/world-news/ukrai...
13,43,2025-10-03-16-17-17 +0000,nypost,Throng of glitzed-up Taylor Swift fans grab ol...,https://nypost.com/2025/10/03/us-news/taylor-s...
115,43,2025-10-03-11-01-00 +0000,startribune,Unemployed Minnesotans face longer job searche...,https://www.startribune.com/long-term-unemploy...
199,40,2025-10-03-01-22-00 +0000,wsj,Senators on both sides of the aisle pressed th...,https://www.wsj.com/politics/national-security...
223,38,2025-10-02-23-22-05 +0000,latimes,Admired climber dies in fall from Yosemite's E...,https://www.latimes.com/california/story/2025-...
47,36,2025-10-03-15-02-10 +0000,nypost,Suspected NJ stalker was driving 70 mph when h...,https://nypost.com/2025/10/03/us-news/suspecte...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
